In [1]:
import os

from langchain_core.messages import HumanMessage
from langchain_openai import AzureChatOpenAI

os.environ["AZURE_OPENAI_API_KEY"] = "38002279fa574fbcba0fbc424ae056d5"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://lumin-azure-openai-rnd.openai.azure.com/"

model = AzureChatOpenAI(
    openai_api_version="2023-03-15-preview",
    azure_deployment='lumin-rnd-gpt-4-32k',
)

In [2]:
from langchain import LLMChain
multi_template = """Answer the following questions one at a time.

Questions:
{questions}

Answers:
"""
long_prompt = PromptTemplate(template=multi_template, input_variables=["questions"])

llm_chain = LLMChain(
    prompt=long_prompt,
    llm=model
)

qs_str = (
    "Which NFL team won the Super Bowl in the 2010 season?\n" +
    "If I am 6 ft 4 inches, how tall am I in centimeters?\n" +
    "Who was the 12th person on the moon?" +
    "How many eyes does a blade of grass have?"
)

print(llm_chain.run(qs_str))